# Combine datasets together

In [1]:
# Import libraries
import os #operating system
import glob # for reading multiple files
from glob import glob
import pandas as pd #pandas for dataframe management
import matplotlib.pyplot as plt #matplotlib for plotting
import matplotlib.dates as mdates # alias for date formatting
import numpy as np # for generating synthetic data

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [2]:
# Set some variables
dataPath = '../data' # set data path
years = [2019]

In [3]:
# Define a function to read the data
def read_data(file):
# Read in excel 
    df = (pd.read_excel(
        file,
        skiprows=range(1,9),
        usecols='B:C',
        header=0,
    ))

    # remove last rows
    df.drop(
        df.tail(2).index,
        inplace=True
    )

    # fix index and naming
    df.columns = ['date', 'demand']
    df['date'] = pd.to_datetime(df['date']) # convert column to datetime
    df.set_index('date', inplace=True)

    return df

# Read all the data

In [4]:
# Create a list of files to combine
PATH = os.path.join(dataPath, str(years[0]))
EXT = "*.xls"
all_files = [file
            for path, subdir, files in os.walk(PATH)
            for file in glob(os.path.join(path, EXT))]

# Assemble files into a final dataframe
df = pd.DataFrame()
for file in all_files:
    tmp = read_data(file)
    dfs = [df, tmp]
    df = pd.concat(dfs)

In [5]:
df.shape

(35381, 1)